In [34]:
import os
import torch
import torchvision
!pip install torchsummary
import torch.nn as nn
from torchvision import models
from torchsummary import summary

In [35]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [36]:
# minibatch size
batch_size = 16

# PreProcessing

In [37]:
from torchvision import transforms
from PIL import Image
import random

# Define resize operation if needed
resize = transforms.Resize((48, 48))  # Replace with (96, 96) if upsizing is desired

# Define transformations for both training and testing
transformations = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    resize,
    transforms.RandomHorizontalFlip(),  # Data augmentation
    transforms.RandomRotation(10),  # Data augmentation
    transforms.RandomResizedCrop((40, 40), scale=(0.8, 1.2), interpolation=Image.BILINEAR),  # Random crop
    #transforms.RandomApply([transforms.RandomErasing()], p=0.5),  # Random erasing
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),  # Normalizing for grayscale image
])

# Use the same transformations for both training and testing datasets
trainTransforms = transformations
testTransforms = transformations

In [38]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import random_split




# Load the dataset from the image folder
dataset = datasets.ImageFolder(root='/kaggle/input/facial-emotion-detection/Facial emotion/train', transform=trainTransforms)

# Calculate the sizes for train and validation sets
train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size

# Split the dataset
train_data, validation_data = random_split(dataset, [train_size, val_size])
# Setup the batch size hyperparameter
BATCH_SIZE = batch_size
test_data = ImageFolder('/kaggle/input/facial-emotion-detection/Facial emotion/test', transform=testTransforms)

# Turn datasets into iterables (batches)
train_dataloader = DataLoader(train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

validation_dataloader = DataLoader(validation_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

test_dataloader = DataLoader(test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False)


# Let's check out what we've created
print(f"Dataloaders: {train_dataloader, test_dataloader}")
print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of Validation dataloader: {len(validation_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")

Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x7b2ce963dcc0>, <torch.utils.data.dataloader.DataLoader object at 0x7b2cf29f68c0>)
Length of train dataloader: 1256 batches of 16
Length of Validation dataloader: 539 batches of 16
Length of test dataloader: 449 batches of 16


In [39]:
nb_train_samples = 1256 * 16
nb_validation_samples = 539 * 16
nb_test_samples = 449 * 16

# **VGG-11**

In [7]:
import torch.nn as nn
import torch.nn.functional as F

class VGGNetFER(nn.Module):
    def __init__(self, num_classes=7):
        super(VGGNetFER, self).__init__()
        # Convolutional blocks
        self.conv_blocks = nn.Sequential(
            # Conv Block 1
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Conv Block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Conv Block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Conv Block 4
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        # Fully connected layers
        self.fc_layers = nn.Sequential(
            nn.Linear(512 * 3 * 3, 4096),  # Adjust the linear layer size based on your conv output
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.conv_blocks(x)
        x = x.view(x.size(0), -1)  # Flatten the output of conv layers
        x = self.fc_layers(x)
        return x  

# Create the model
num_classes = 7  # for FER2013 dataset
model = VGGNetFER(num_classes)


# **RESNET-34**

In [40]:
class BlockBuilder(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BlockBuilder, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        identity = self.shortcut(x)
        x = torch.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += identity
        x = torch.relu(x)
        return x


class ResNet_model(nn.Module):
    def __init__(self, block, num_blocks):
        super(ResNet_model, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.residual_layer1 = self._make_residual_layer(block, 64, num_blocks[0], stride=1)
        self.residual_layer2 = self._make_residual_layer(block, 128, num_blocks[1], stride=2)
        self.residual_layer3 = self._make_residual_layer(block, 256, num_blocks[2], stride=2)
        self.residual_layer4 = self._make_residual_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  
        self.linear = nn.Linear(512*block.expansion,36)

    def _make_residual_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = self.residual_layer1(x)
        x = self.residual_layer2(x)
        x = self.residual_layer3(x)
        x = self.residual_layer4(x)
        x = self.avgpool(x) 
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

def ResNet34():
    return ResNet_model(BlockBuilder, [3, 4, 6, 3])

model = ResNet34().to(device)
print(model)


ResNet_model(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (residual_layer1): Sequential(
    (0): BlockBuilder(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BlockBuilder(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, mo

In [41]:
import torch
import torch.optim as optim
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Criterion and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, nesterov=True)

# Scheduler
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=0.75, patience=5, verbose=True)

num_epochs = 25
model = model.to(device)

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_dataloader.dataset)
    epoch_acc = running_corrects.double() / len(train_dataloader.dataset)
    print(f'Epoch {epoch}/{num_epochs - 1} - Training loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

    # Validation phase
    model.eval()
    val_running_loss = 0.0
    val_running_corrects = 0
    with torch.no_grad():
        for inputs, labels in validation_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            val_running_loss += loss.item() * inputs.size(0)
            val_running_corrects += torch.sum(preds == labels.data)

    val_epoch_loss = val_running_loss / len(validation_dataloader.dataset)
    val_epoch_acc = val_running_corrects.double() / len(validation_dataloader.dataset)
    print(f'Epoch {epoch}/{num_epochs - 1} - Validation loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_acc:.4f}')

    # Adjust learning rate based on validation accuracy
    scheduler.step(val_epoch_acc)

# Save the model after training
#torch.save(model.state_dict(), 'vggnet_fer.pth')


Epoch 0/24 - Training loss: 1.7165, Accuracy: 0.3160
Epoch 0/24 - Validation loss: 1.5676, Accuracy: 0.3834
Epoch 1/24 - Training loss: 1.5378, Accuracy: 0.4038
Epoch 1/24 - Validation loss: 1.4764, Accuracy: 0.4327
Epoch 2/24 - Training loss: 1.4505, Accuracy: 0.4350
Epoch 2/24 - Validation loss: 1.3887, Accuracy: 0.4637
Epoch 3/24 - Training loss: 1.3818, Accuracy: 0.4677
Epoch 3/24 - Validation loss: 1.3442, Accuracy: 0.4802
Epoch 4/24 - Training loss: 1.3202, Accuracy: 0.4938
Epoch 4/24 - Validation loss: 1.3018, Accuracy: 0.4999
Epoch 5/24 - Training loss: 1.2724, Accuracy: 0.5135
Epoch 5/24 - Validation loss: 1.2806, Accuracy: 0.5138
Epoch 6/24 - Training loss: 1.2357, Accuracy: 0.5307
Epoch 6/24 - Validation loss: 1.2619, Accuracy: 0.5169
Epoch 7/24 - Training loss: 1.2013, Accuracy: 0.5459
Epoch 7/24 - Validation loss: 1.2403, Accuracy: 0.5313
Epoch 8/24 - Training loss: 1.1631, Accuracy: 0.5565
Epoch 8/24 - Validation loss: 1.2043, Accuracy: 0.5457
Epoch 9/24 - Training loss: 

In [42]:
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
test_corrects = 0

# You should define nb_test_samples before this block
# It should be the total number of samples in the test set
nb_test_samples = len(test_dataloader.dataset)

with torch.no_grad():
    for inputs, labels in test_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)

        _, preds = torch.max(outputs, 1)
        test_corrects += torch.sum(preds == labels.data).item()

# Calculate the average loss and accuracy
test_epoch_loss = test_loss / nb_test_samples
test_epoch_accuracy = test_corrects / nb_test_samples
test_accuracy_percentage = test_epoch_accuracy * 100

print(f'Test loss: {test_epoch_loss:.4f}')
print(f'Test Accuracy: {test_accuracy_percentage:.2f}%')


Test loss: 1.2333
Test Accuracy: 56.62%
